In [27]:
import numpy as np
import cv2
import os

In [28]:
params_dir = os.getcwd()+'/camera_params/monocular_camera_params/'
print(params_dir)

#load camera parameters
K = np.load(params_dir+'K.npy')
dist = np.load(params_dir+'dist.npy')

print("Camera matrix")
print(K)
print("Len distortion")
print(dist)

chessboard_size = (8,6) # Your chessboard size
# iterative termination criteria, maximum iterationm and epsilon
term_criteria = (cv2.TermCriteria_EPS+ cv2.TermCriteria_MAX_ITER, 30, 0.001)
# Defining the world coordinates for 3D points
objp = np.zeros((1, chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
# 3D object for visualization
axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)
box = np.float32([[0,0,0], [0,4,0], [4,4,0], [4,0,0],
                   [0,0,-4],[0,4,-4],[4,4,-4],[4,0,-4] ])

d:\KMUTNB\IEE\Image Processing\010723305-Computer-Vision/camera_params/monocular_camera_params/
Camera matrix
[[1.29968396e+03 0.00000000e+00 6.32814378e+02]
 [0.00000000e+00 1.29933998e+03 4.74721903e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Len distortion
[[-0.40795692  0.24741507  0.00200567 -0.00202735 -0.09445672]]


In [19]:
def draw_axis(img, corners, imgpts) :
    corner = tuple(corners[0].ravel().astype(int))
    img = cv2.line(img, corner, tuple(imgpts[0].ravel().astype(int)), (255,0,0), 10)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel().astype(int)), (0,255,0), 10)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel().astype(int)), (0,0,255), 10)
    return img

In [20]:
def draw_box(img, corners, imgpts) :
    imgpts = np.int32(imgpts).reshape(-1,2)
    # draw ground floor in green
    img = cv2.drawContours(img, [imgpts[:4]],-1,(0,255,0),-3)
    # draw pillars in blue color
    for i,j in zip(range(4),range(4,8)):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]),(255),10)
    # draw top layer in red color
    img = cv2.drawContours(img, [imgpts[4:]],-1,(0,0,255),10)
    return img

In [21]:
def augmented_image(frame,im_src, pts_src, pts_dst):
    
    # Calculate Homography
    h, status = cv2.findHomography(pts_src, pts_dst)

    # Warp source image to destination based on homography
    warped_image = cv2.warpPerspective(im_src, h, (frame.shape[1],frame.shape[0]))
            
    # Prepare a mask representing region to copy from the warped image into the original frame.
    mask = np.zeros([frame.shape[0], frame.shape[1]], dtype=np.uint8)
    cv2.fillConvexPoly(mask, np.int32(pts_dst), (255, 255, 255), cv2.LINE_AA)
    im_out = cv2.add(frame, warped_image, mask=cv2.bitwise_not(mask))
    im_out = cv2.add(im_out, warped_image)
    
    return im_out

<h2 style="text-align: center">
    <a href = "https://docs.opencv.org/4.5.3/d9/d0c/group__calib3d.html#ga549c2075fac14829ff4a58bc931c033d"> solvePNP </a>
    is the function that finds an object pose from 3D-2D point correspondences. <br>
    <a href = "https://docs.opencv.org/4.5.3/d9/d0c/group__calib3d.html#ga1019495a2c8d1743ed5cc23fa0daff8c"> projectPoints </a>
    is the function that The function computes the 2D projections of 3D points to the image plane. <br>
</h2>

$$ x = PX\ where\ P\ is\ the\ camera\ projection\ matrix$$


In [22]:
# cap = cv2.VideoCapture(2,cv2.CAP_DSHOW)
cap = cv2.VideoCapture('./videos/monocular/chess_board.mp4')
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
vdo = cv2.VideoCapture('./videos/monocular/aruco.mp4')
_, im_src = vdo.read()
# im_src = cv2.imread('./images/Grumpy-cat.jpg')
im_src_size = im_src.shape[:2]
src_points = np.float32([[0,0], [im_src_size[1],0],[im_src_size[1], im_src_size[0]] ,[0, im_src_size[0]] ])
while cap.isOpened() :
    ret1, frame = cap.read()
    ret2, im_src = vdo.read()
    if ret1 and ret2 :
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        img = frame.copy()
        found, corners = cv2.findChessboardCorners(gray, chessboard_size,None)
        if found :
            corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),term_criteria)
            
            dst_points = np.float32([corners2[0], corners2[7], corners2[47], corners2[40]])
            # Find the rotation and translation vectors.
            ret,rvecs, tvecs = cv2.solvePnP(objp, corners2, K, dist)
            # project 3D points to image plane
            axis_imgpts, _ = cv2.projectPoints(axis, rvecs, tvecs, K, dist)
            box_imgpts, _ = cv2.projectPoints(box, rvecs, tvecs, K, dist)
            
            augmented = augmented_image(img, im_src, src_points, dst_points)
            img = cv2.drawChessboardCorners(frame, chessboard_size, corners2, ret)
            # img = draw_axis(frame,corners2,axis_imgpts)
            # img = draw_box(img, corners, box_imgpts)
            cv2.imshow('augmented', augmented)
            
    
        cv2.imshow('Chessboard Pose estimation', img)
        if cv2.waitKey(33) & 0xff == 27 :
            break
    else :
        break

cv2.destroyAllWindows()
cap.release()

<h1 style = "text-align: center">
    <a href="http://www.uco.es/investiga/grupos/ava/node/26"> ArUco</a> is that stand for "Augmented Reality University of Cordoba" <br>
    is an opensource project for augmented reality and image-based object pose estimation <br>
    For this project, it has been built into the open CV as well. It can be referenced from the following 
    <a href = "https://docs.opencv.org/4.5.3/d9/d6a/group__aruco.html"> ArUco Marker Detection </a>
</h1>

In [23]:
params_dir = os.getcwd()+'/camera_params/new-camera_params/'
print(params_dir)

#load camera parameters
K = np.load(params_dir+'K.npy')
dist = np.load(params_dir+'dist.npy')

print("Camera matrix")
print(K)
print("Len distortion")
print(dist)

d:\KMUTNB\IEE\Image Processing\010723305-Computer-Vision/camera_params/new-camera_params/
Camera matrix
[[1.29968396e+03 0.00000000e+00 6.32814378e+02]
 [0.00000000e+00 1.29933998e+03 4.74721903e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Len distortion
[[-0.40795692  0.24741507  0.00200567 -0.00202735 -0.09445672]]


In [24]:
AruCo_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
AruCo_params = cv2.aruco.DetectorParameters_create()
board = cv2.aruco.GridBoard_create(3, 4, 0.05, 0.0075, AruCo_dict)

In [25]:
def write_text(img, pose, dy, text) :
    x0 = pose[0]
    y0 = pose[1]
    for i, line in enumerate(text.split('\n')) :
        y = y0 + i*dy
        cv2.putText(img, line, np.int32([x0, y]), cv2.FONT_HERSHEY_COMPLEX, 0.75, (50,200,255), 2)

In [26]:
# cap = cv2.VideoCapture(2,cv2.CAP_DSHOW)
cap = cv2.VideoCapture('./videos/monocular/aruco.mp4')
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while cap.isOpened() :
    ret, frame = cap.read()
    if ret :
        img = frame.copy()
        
        markerCorners, markerIds, rejectedCandidates = cv2.aruco.detectMarkers(frame, AruCo_dict, parameters = AruCo_params)
        
        if len(markerCorners) > 0:
            img = cv2.aruco.drawDetectedMarkers(frame, markerCorners)
            rvecs, tvecs, points = cv2.aruco.estimatePoseSingleMarkers(markerCorners , 0.05, K, dist)
            for (rvec, tvec, id, corner) in zip(rvecs, tvecs, markerIds, markerCorners) :
                img = cv2.aruco.drawAxis(frame, K, dist, rvec, tvec, 0.05)
                x = tvec[0,0]
                y = tvec[0,1]
                z = tvec[0,2]
                text = "id: {}\n pose:\n {:.3f}\n {:.3f}\n {:.3f}".format(id, x, y, z)
                cX = (corner[0,0][0] + corner[0,2][0]) / 2
                cY = (corner[0,0][1] + corner[0,2][1]) / 2
                write_text(img, (cX, cY), 20, text)
            ret, brvec, btvec = cv2.aruco.estimatePoseBoard(markerCorners, markerIds, board, K, dist, rvecs, tvecs) 
            if ret :
                img = cv2.aruco.drawAxis(frame, K, dist, brvec, btvec, 0.05)
    
        cv2.imshow('Frame', img)
        if cv2.waitKey(1) & 0xff == 27 :
            break
    else :
        break
    

cv2.destroyAllWindows()
cap.release()